In [15]:
import csv
from io import StringIO
import itertools
import time
import json
import numpy as np


In [16]:
#Tree class
class CreateTree:
    def __init__(self, data):
        self.data = data
        self.count = data
        self.childrenNode = []
        self.parentNode = None

    def level(self):
        level = 0
        p = self.parentNode
        while p:
            level += 1
            p = p.parentNode

        return level

    def printTree(self):
        spaces = ' ' * self.level() * 3
        prefix = spaces + "= " if self.parentNode else ""
        print(prefix + self.data)
        if self.childrenNode:
            for child in self.childrenNode:
                child.printTree()

    def addChild(self, child):
        child.parentNode = self
        self.childrenNode.append(child)



In [17]:
#Creates the Tree using the sorted transaction list
def buildTree(sortedTransationList):
    root = CreateTree("Null")
    count = 1
    print()
    for i in range(len(sortedTransationList)):
        parent = CreateTree(sortedTransationList[i][0])       
        for j in range(1,len(sortedTransationList[i])):
                parent.addChild(CreateTree(sortedTransationList[i][j]))
        root.addChild(parent)
    root.printTree()


In [18]:
#Sorts every transaction in the dataset based on the sorted unique Item Set
sortedTransationList =[]
def sortTransaction(dataset,sortItemSet):
    for i in range(len(dataset)):
        dataset[i].remove("")
        so_li = {v:j for j, v in enumerate(sortItemSet)}
        sorted_list = sorted(dataset[i], key=lambda v: so_li[v]) 
        sortedTransationList.append(sorted_list)
    buildTree(sortedTransationList)    
    return 0


In [19]:
#Sorts the unique Item Set
def sortOneItemSet(itemSet,dataset):
    sortItemSet = sorted(itemSet, key=lambda x: x[0], reverse=True)
    temCount,sortItemSet = map(list, zip(*sortItemSet))
    print()
    print("Sorted Item Set : " +str(sortItemSet))
    print()
    sortTransaction(dataset,sortItemSet)
    return sortItemSet


In [20]:
#generates subsets of the frequent itemSets
#finds the support count and confidence of the subsets and checks with the minimum confidence to find the association rules 
def association(itemSet,dataset,originalDataset,x,m):
    supportList =[]
    subCountList =[]
    subList = []
    associationList =[]
    if(x != 1):
        for i in range(len(itemSet)):
            newList =[]
            for j in range(1,1<<len(itemSet[i][1])):
                newList.append([itemSet[i][1][l] for l in range(len(itemSet[i][1])) if (j & (1 << l))])
            subList.append(newList)
        print()    
        for i in range(len(subList)):
            subCountList = itemCount(subList[i],dataset,originalDataset,x,7) 
            for j in range(len(subCountList)-1):
                k = len(subCountList)-1
                confidence = (subCountList[k][0]/subCountList[j][0])*100    
                if(confidence > min_confidence):
                    y = list(np.setdiff1d([subCountList[k][1]],[subCountList[j][1]]))
                    association = list([subCountList[j][1]]+ [y])
                    associationList.append(association)
             
        with open("Answers.txt",'a') as f:
            print("Association Rules for "+str(x)+" item Set ") 
            f.writelines(json.dumps("Association Rules for "+str(x)+" item Set "))
            f.write("\n")
            if(len(associationList) == 0):
                f.writelines(json.dumps("No Association Rules"))
                f.write("\n")
                print("No Association Rules")
                f.write("\n")
            else: 
                f.writelines(json.dumps(associationList))
                f.write("\n")
                print(associationList)
                f.write("\n")
    return 0


In [21]:
#removes rows which has a suport count less than the minimum support count
def aprioriDeleteRow(uniqueItems,dataset,originalDataset,x,m):
    lessIndex =[]
    lessItem = []
    for i in range (len(uniqueItems)):
        if (uniqueItems[i][0] < min_sup) :
                lessItem.append(i)
                if(x == 1):
                    for j in range(len(dataset)):
                        for k in range(len(dataset[j])):
                            if(uniqueItems[i][1] == (dataset[j][k])):
                                lessIndex.append(j)
                else:
                    for j in range(len(dataset)):
                        set2 = set(dataset[j])
                        set1 = set(uniqueItems[i][1])
                        if(set1.issubset(set2)):
                            lessIndex.append(j)
                    
    #creates a new dataset removing items less than support count
    dataset = [i for j, i in enumerate(dataset) if j not in lessIndex]
    #removes items from the uniquelist which are less than support count
    uniqueItems = [i for j, i in enumerate(uniqueItems) if j not in lessItem] 
    
    if(uniqueItems):
        itemCount,itemSet = map(list, zip(*uniqueItems))
        with open("Answers.txt",'a') as f:
            f.writelines(json.dumps("Apriori Frequent "+str(x)+" ItemSet : "+str(len(itemSet))))
            f.write("\n")
            f.writelines(json.dumps(itemSet))
            f.write("\n")
            f.write("\n")
        association(uniqueItems,dataset,originalDataset,x,m)
        x +=1
        aprioriItemSet(x,itemSet,dataset,originalDataset,m)    
    else:
        print("End of list")


In [22]:
#generates apriori item sets 
def aprioriItemSet(x,itemSet,dataset,originalDataset,m):
    if(x == 2):  #for 2 item Set
        combinations = list(itertools.combinations(itemSet, x))
        itemCount(combinations,dataset,originalDataset,x,m)
    else:
        new_ItemSet = []   
        for i in range(len(itemSet)):    
            for j in range(len(itemSet)):
                if(i != j): 
                    for k in range(len(itemSet[j])):
                        if(itemSet[j][k] not in itemSet[i]):   
                            new_set = list(itemSet[i])+[itemSet[j][k]]
                            new_set.sort()
                            if(new_set not in new_ItemSet):
                                new_ItemSet.append(new_set)
        itemCount(new_ItemSet,dataset,originalDataset,x,m)


In [23]:
    
#generates frequent item Sets for bruteforce approach
def frequentItemSet(itemSet,dataset,originalDataset,x,m):
    lessFreq =[]
    for i in range(len(itemSet)):
        if(itemSet[i][0] < min_sup):
            lessFreq.append(i)

    #creates a new list of a dataset removing items less than support count
    itemSet = [i for j, i in enumerate(itemSet) if j not in lessFreq]
    with open("Answers.txt",'a') as f:
        f.writelines(json.dumps("Bruteforce Frequent "+str(x)+" ItemSet : "+str(len(itemSet))))
        f.write("\n")
        f.writelines(json.dumps(itemSet))
        f.write("\n")
        f.write("\n")
    association(itemSet,dataset,originalDataset,x,m)
    return 0

In [24]:
#find all the itemSet of the given number(2-5) for bruteforce method
def allItemSet(uniqueItems,dataset,i,m):
        uniqueItemsSet =[]
        combinations = list(itertools.combinations(uniqueItems, i))
        itemCount(combinations,dataset,dataset,i,m)   # sends the item sets to get the count of each sets 


In [25]:
#find the count of items from the original dataset.    
def itemCount(combinations,dataset,originalDataset,x,m):
    countList =[]
    for c in range(len(combinations)):
        count = 0
        if(x==1):
            for i in range(len(originalDataset)):
                for j in range(len(originalDataset[i])):
                    if(originalDataset[i][j] == uniqueItems[c]):
                        count += 1
            countList.append(count)
        else:
            for i in range(len(originalDataset)):
                set1 = set(combinations[c])
                set2 = set(originalDataset[i])       
                if(set1.issubset(set2)):    
                    count += 1
            countList.append(count) 
    itemSet = list(zip(countList,combinations))
    if(m == 1):
        frequentItemSet(itemSet,dataset,originalDataset,x,m)
    if(m == 2):
        aprioriDeleteRow(itemSet,dataset,originalDataset,x,m)
    if(m == 3):
        sortOneItemSet(itemSet,dataset)
    if(m == 7):    #when generating association rules
        return itemSet


In [26]:
#finds the unique items from the dataset        
uniqueItems = []  
def getUniqueItems(dataset,m):
    for i in range(len(dataset)):
        for j in range(len(dataset[i])):
                if(dataset[i][j] in uniqueItems):
                    break
                else:
                    uniqueItems.append(dataset[i][j])                
    if (m == 1):  # for bruteforce Algorithm
        uniqueItems.remove("")
        itemCount(uniqueItems,dataset,dataset,1,m) 
        for i in range(2,6):
            allItemSet(uniqueItems,dataset,i,m)  #sends the unique itemset to find the other itemset(2-5)
            continue   
    if(m ==2):  #for Apriori algorithm
        aprioriItemSet(2,uniqueItems,dataset,dataset,m)
    if(m ==3):
        itemCount(uniqueItems,dataset,dataset,1,m)


In [27]:
#finds the run time of the algorithms        
def runTime():
    for m in range(1,4): 
        start = time.process_time()
        if(m==1):
            name = "BruteForce"
        if(m==2):
            name = "Apriori "
        if(m==3):
            name = "FP Growth "
        getUniqueItems(fullDataset,m)
        with open("Answers.txt",'a') as f:    
            f.writelines(json.dumps("Run Time of "+name+" "+ str(time.process_time() - start)))
            f.write("\n")
            f.write("\n")
            f.write("\n")
            print("Run Time of "+name+" "+ str(time.process_time() - start))        


In [28]:
#read data from the dataset and stores in a list
file = open("GroceryStore.csv", 'r')
readFile = csv.reader(file, quotechar='\\')
fullDataset =[]
for row in readFile:
    fullDataset.append(row)
    
#get the minimum support count    
min_sup = int(input("Enter minimum support count :"))
#get the minimum confidence 
min_confidence = int(input("Enter minimum confidence :"))
            
if __name__ == '__main__':
    runTime()    
            

Enter minimum support count : 1250
Enter minimum confidence : 50



Association Rules for 2 item Set 
No Association Rules

Association Rules for 3 item Set 
[[['Lassi', 'Panner'], ['Sweet']], [['Sweet', 'Panner'], ['Lassi']]]

Association Rules for 4 item Set 
No Association Rules

Association Rules for 5 item Set 
No Association Rules
Run Time of BruteForce 11.161113999999998

Association Rules for 2 item Set 
No Association Rules

Association Rules for 3 item Set 
[[['Lassi', 'Panner'], ['Sweet']], [['Panner', 'Sweet'], ['Lassi']]]
End of list
Run Time of Apriori  17.018062

Sorted Item Set : ['Milk', 'Ghee', 'Coffee Powder', 'Yougurt', 'Bread', 'Sweet', 'Sugar', 'Butter', 'Cheese', 'Panner', 'Lassi', 'Tea Powder']


Null
   = Ghee
      = Coffee Powder
      = Yougurt
      = Butter
      = Cheese
      = Lassi
   = Ghee
      = Coffee Powder
   = Butter
      = Cheese
      = Lassi
      = Tea Powder
   = Coffee Powder
      = Bread
      = Butter
      = Cheese
      = Panner
      = Tea Powder
   = Coffee Powder
      = Yougurt
      = Sweet
  